In [1]:
import pandas as pd
import time
import requests
import json

def get_cards():
    all_cards = []
    page = 1
    
    while True:
        # Search for cards legal in Commander
        url = f"https://api.scryfall.com/cards/search?q=legal:commander&page={page}"
        response = requests.get(url)
        
        if response.status_code != 200:
            break
            
        data = response.json()
        all_cards.extend(data['data'])
        
        print(f"Page {page}: Got {len(data['data'])} cards. Total so far: {len(all_cards)}")
        
        if not data.get('has_more'):
            break
            
        page += 1
        time.sleep(0.02)  # Be nice to the API
    
    return all_cards

# This might take a minute or two
cards_data = get_cards()
print(f"Total cards collected: {len(cards_data)}")

Page 1: Got 175 cards. Total so far: 175
Page 2: Got 175 cards. Total so far: 350
Page 3: Got 175 cards. Total so far: 525
Page 4: Got 175 cards. Total so far: 700
Page 5: Got 175 cards. Total so far: 875
Page 6: Got 175 cards. Total so far: 1050
Page 7: Got 175 cards. Total so far: 1225
Page 8: Got 175 cards. Total so far: 1400
Page 9: Got 175 cards. Total so far: 1575
Page 10: Got 175 cards. Total so far: 1750
Page 11: Got 175 cards. Total so far: 1925
Page 12: Got 175 cards. Total so far: 2100
Page 13: Got 175 cards. Total so far: 2275
Page 14: Got 175 cards. Total so far: 2450
Page 15: Got 175 cards. Total so far: 2625
Page 16: Got 175 cards. Total so far: 2800
Page 17: Got 175 cards. Total so far: 2975
Page 18: Got 175 cards. Total so far: 3150
Page 19: Got 175 cards. Total so far: 3325
Page 20: Got 175 cards. Total so far: 3500
Page 21: Got 175 cards. Total so far: 3675
Page 22: Got 175 cards. Total so far: 3850
Page 23: Got 175 cards. Total so far: 4025
Page 24: Got 175 cards. T

In [2]:
# Create DataFrame with the fields we need
df = pd.DataFrame(cards_data)

# Remove cards without oracle text (lands, some artifacts)
df_clean = df[df['oracle_text'].notna()].copy()

# Add a combined text column for ML processing
df_clean['combined_text'] = (
    df_clean['oracle_text'].fillna('') + ' ' + 
    df_clean['type_line'].fillna('') + ' ' +
    df_clean['keywords'].astype(str)
)

print(f"DataFrame shape: {df.shape}")
print(f"Columns available: {df.columns.tolist()}")

DataFrame shape: (26775, 79)
Columns available: ['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors', 'color_identity', 'keywords', 'legalities', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'edhrec_rank', 'preview', 'prices', 'related_uris', 'purchase_uris', 'power', 'toughness', 'all_parts', 'frame_effects', 'security_stamp', 'penny_rank', 'promo_types', 'produced_mana', 'mtgo_foil_id', 

In [4]:
# Save to file
import os

os.makedirs('data', exist_ok=True)
df.to_csv('data/mtg_cards_raw.csv', index=False)
print(f"Saved {len(cards_data)} raw cards to data/mtg_cards_raw.csv")
df_clean.to_csv('data/mtg_cards_clean.csv', index=False)
print(f"Saved {len(df_clean)} cleaned cards to data/mtg_cards_clean.csv")

Saved 26775 raw cards to data/mtg_cards_raw.csv
Saved 26092 cleaned cards to data/mtg_cards_clean.csv


In [14]:
# Use below to not reload the whole data set again but edit df_clean
import pandas as pd
df = pd.read_csv('data/mtg_cards_raw.csv', low_memory=False)

# More thorough cleaning
def clean_oracle_text(df):
    df_clean = df.copy()
    
    # Remove rows where oracle_text is NaN
    df_clean = df_clean[df_clean['oracle_text'].notna()]
    
    # Remove rows where oracle_text is not a string
    df_clean = df_clean[df_clean['oracle_text'].apply(lambda x: isinstance(x, str))]
    
    # Remove rows where oracle_text is empty string or just whitespace
    df_clean = df_clean[df_clean['oracle_text'].str.strip() != '']
    
    # Remove rows where oracle_text is 'nan' as a string
    df_clean = df_clean[df_clean['oracle_text'].str.lower() != 'nan']
    
    return df_clean

df_clean = clean_oracle_text(df)

# Add a combined text column for ML processing
df_clean['combined_text'] = (
    df_clean['oracle_text'].fillna('') + ' ' + 
    df_clean['type_line'].fillna('') + ' ' +
    df_clean['keywords'].astype(str)
)

In [16]:
df_clean.to_csv('data/mtg_cards_clean.csv', index=False)